## Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Read Data & Preprocessing

In [3]:
from sklearn.preprocessing import StandardScaler

In [20]:
path = "./000001.SS.csv"
dt = pd.read_csv(path)
dt

tar_var = dt['Close'].var()
tar_mean= dt['Close'].mean()
tar_var
tar_mean

dt = dt.drop(columns=['Date'])
dt[dt.columns] = StandardScaler().fit_transform(dt[dt.columns])
dt

,Date,Open,High,Low,Close,Volume
0,2019/4/8,3271.271973,3288.452881,3210.514893,3244.810059,504500
1,2019/4/9,3240.394043,3251.645996,3215.705078,3239.664063,364200
2,2019/4/10,3224.645020,3264.778076,3204.878906,3241.929932,380100
3,2019/4/11,3241.547119,3254.125977,3185.545898,3189.961914,353000
4,2019/4/12,3184.860107,3199.278076,3168.041992,3188.625977,289800
...,...,...,...,...,...,...
478,2021/3/29,3429.632080,3449.833984,3409.886963,3435.300049,284800
479,2021/3/30,3432.530029,3457.629883,3423.320068,3456.679932,285400
480,2021/3/31,3452.209961,3452.209961,3420.830078,3441.909912,283000
481,2021/4/1,3444.810059,3470.030029,3438.830078,3466.330078,275200


61767.19842913412

3111.2904398902706

,Open,High,Low,Close,Volume
0,0.650349,0.632000,0.498450,0.537794,2.759595
1,0.526316,0.485507,0.519671,0.517067,1.219429
2,0.463054,0.537773,0.475406,0.526194,1.393974
3,0.530948,0.495378,0.396359,0.316875,1.096480
4,0.303242,0.277082,0.324791,0.311494,0.402692
...,...,...,...,...,...
478,1.286464,1.274301,1.313621,1.305056,0.347803
479,1.298104,1.305329,1.368545,1.391170,0.354390
480,1.377156,1.283757,1.358364,1.331679,0.328043
481,1.347432,1.354682,1.431961,1.430039,0.242418


## Time Series Preprocessing -- Lags & Shifts

In [23]:
dt.shape

(483, 5)

In [26]:
# parameters
tr_len = 30
ts_len = 5
X_arr = []
y_arr = []

for idx in range(dt.shape[0] - tr_len - ts_len + 1):
    X_arr.append(dt.iloc[range(idx, idx + tr_len)]) # No.tr_len of input
    y_arr.append(dt['Close'].iloc[range(idx + tr_len, idx + tr_len + ts_len)]) # tr_len
#X_arr
y_arr[-1]
len(X_arr)

478    1.305056
479    1.391170
480    1.331679
481    1.430039
482    1.502790
Name: Close, dtype: float64

449

In [42]:
# split train & test
x_train = np.array(X_arr[:-ts_len])
y_train = np.array(y_arr[:-ts_len])
x_test = np.array([np.array(X_arr[-1])])
y_test = np.array([np.array(y_arr[-1])])

In [43]:
x_train.shape
x_test.shape
y_train.shape
y_test.shape

(444, 30, 5)

(1, 30, 5)

(444, 5)

(1, 5)

## RNN & LSTM

## 1. LSTM 

In [35]:
import tensorflow as tf
from keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError
from tensorflow.keras.models import Sequential

In [78]:
# LSTM
def get_model(params, input_shape):
    model = Sequential()
    model.add(LSTM(units=params["lstm_units"], return_sequences=True, input_shape=(input_shape, 5)))
    #model.add(Dropout(rate=params["dropout"]))

    model.add(LSTM(units=params["lstm_units"], return_sequences=True))
    #model.add(Dropout(rate=params["dropout"]))

    model.add(LSTM(units=params["lstm_units"], return_sequences=True))
    #model.add(Dropout(rate=params["dropout"]))

    model.add(LSTM(units=params["lstm_units"], return_sequences=False))
    #model.add(Dropout(rate=params["dropout"]))

    model.add(Dense(5))

    model.compile(loss=params["loss"],
                  optimizer=params["optimizer"],
                  metrics=[RootMeanSquaredError(), MeanAbsoluteError()]
                 )

    return model

In [79]:
params = {
    "loss": "mean_squared_error",
    "optimizer": "adam",
    #"dropout": 0.2,
    "lstm_units": 90,
    "epochs": 60,
    "batch_size": 64,
    "es_patience" : 10
}

model = get_model(params=params, input_shape=tr_len)
model.fit(
    x_train,
    y_train,
    #validation_data=(x_test, y_test),
    epochs=params["epochs"],
    batch_size=params["batch_size"],
    verbose=1#,
    #callbacks=[neptune_callback, es_callback]
)


Epoch 1/60
7/7 [==============================] - 6s 68ms/step - loss: 0.5575 - root_mean_squared_error: 0.7467 - mean_absolute_error: 0.6039
Epoch 2/60
7/7 [==============================] - 0s 60ms/step - loss: 0.2678 - root_mean_squared_error: 0.5175 - mean_absolute_error: 0.3824
Epoch 3/60
7/7 [==============================] - 0s 57ms/step - loss: 0.2270 - root_mean_squared_error: 0.4765 - mean_absolute_error: 0.3546
Epoch 4/60
7/7 [==============================] - 0s 57ms/step - loss: 0.2013 - root_mean_squared_error: 0.4487 - mean_absolute_error: 0.3199
Epoch 5/60
7/7 [==============================] - 0s 58ms/step - loss: 0.1763 - root_mean_squared_error: 0.4199 - mean_absolute_error: 0.3057
Epoch 6/60
7/7 [==============================] - 0s 57ms/step - loss: 0.1570 - root_mean_squared_error: 0.3962 - mean_absolute_error: 0.2835
Epoch 7/60
7/7 [==============================] - 0s 57ms/step - loss: 0.1436 - root_mean_squared_error: 0.3789 - mean_absolute_error: 0.2711
Epoch 

7/7 [==============================] - 0s 63ms/step - loss: 0.0300 - root_mean_squared_error: 0.1732 - mean_absolute_error: 0.1325
Epoch 59/60
7/7 [==============================] - 0s 65ms/step - loss: 0.0302 - root_mean_squared_error: 0.1739 - mean_absolute_error: 0.1311
Epoch 60/60
7/7 [==============================] - 0s 62ms/step - loss: 0.0274 - root_mean_squared_error: 0.1654 - mean_absolute_error: 0.1232


In [80]:
y_pred = model.predict(x_test)
y_pred

array([[1.1935788, 1.059907 , 0.9838619, 1.000453 , 1.036655 ]],
      dtype=float32)

In [81]:
y_true = y_test

In [82]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(y_pred, y_true, squared=False)
rmse

0.337255572595044

### LSTM: Training RMSE is 0.1654, Testing RMSE is 0.3373

In [83]:
# mape
np.mean(np.abs((y_true - y_pred) / y_true))

0.23906113157626793

## 2. RNN

In [57]:
from tensorflow.keras.layers import SimpleRNN

In [58]:
# RNN
def rnn_model(params, input_shape):
    model = Sequential()
    model.add(SimpleRNN(units=params["lstm_units"], return_sequences=True, input_shape=(input_shape, 5)))
    #model.add(Dropout(rate=params["dropout"]))

    model.add(SimpleRNN(units=params["lstm_units"], return_sequences=True))
    #model.add(Dropout(rate=params["dropout"]))

    model.add(SimpleRNN(units=params["lstm_units"], return_sequences=True))
    #model.add(Dropout(rate=params["dropout"]))

    model.add(SimpleRNN(units=params["lstm_units"], return_sequences=False))
    #model.add(Dropout(rate=params["dropout"]))

    model.add(Dense(5))

    model.compile(loss=params["loss"],
                  optimizer=params["optimizer"],
                  metrics=[RootMeanSquaredError(), MeanAbsoluteError()]
                 )

    return model

In [59]:
mdl = rnn_model(params=params, input_shape=tr_len)
mdl.fit(
    x_train,
    y_train,
    #validation_data=(x_test, y_test),
    epochs=params["epochs"],
    batch_size=params["batch_size"],
    verbose=1#,
    #callbacks=[neptune_callback, es_callback]
)



Epoch 1/60
7/7 [==============================] - 2s 20ms/step - loss: 0.5007 - root_mean_squared_error: 0.7076 - mean_absolute_error: 0.5130
Epoch 2/60
7/7 [==============================] - 0s 17ms/step - loss: 0.1519 - root_mean_squared_error: 0.3898 - mean_absolute_error: 0.2904
Epoch 3/60
7/7 [==============================] - 0s 16ms/step - loss: 0.1151 - root_mean_squared_error: 0.3393 - mean_absolute_error: 0.2493
Epoch 4/60
7/7 [==============================] - 0s 16ms/step - loss: 0.0912 - root_mean_squared_error: 0.3020 - mean_absolute_error: 0.2225
Epoch 5/60
7/7 [==============================] - 0s 16ms/step - loss: 0.0823 - root_mean_squared_error: 0.2868 - mean_absolute_error: 0.2092
Epoch 6/60
7/7 [==============================] - 0s 16ms/step - loss: 0.0776 - root_mean_squared_error: 0.2785 - mean_absolute_error: 0.2033
Epoch 7/60
7/7 [==============================] - 0s 16ms/step - loss: 0.0718 - root_mean_squared_error: 0.2679 - mean_absolute_error: 0.1969
Epoch 

7/7 [==============================] - 0s 17ms/step - loss: 0.0155 - root_mean_squared_error: 0.1244 - mean_absolute_error: 0.0950
Epoch 59/60
7/7 [==============================] - 0s 17ms/step - loss: 0.0146 - root_mean_squared_error: 0.1207 - mean_absolute_error: 0.0928
Epoch 60/60
7/7 [==============================] - 0s 18ms/step - loss: 0.0156 - root_mean_squared_error: 0.1247 - mean_absolute_error: 0.0965


In [60]:
y_pred = mdl.predict(x_test)
y_pred
y_true = y_test#.to_numpy()
y_true
rmse = mean_squared_error(y_pred, y_true, squared=False)
rmse

array([[1.1955949, 1.1364983, 1.1938225, 1.2700427, 1.2297664]],
      dtype=float32)

array([[1.30505551, 1.39117004, 1.33167891, 1.43003918, 1.50278989]])

0.1870017521757325

### For RNN, Training RMSE is 0.1247, Testing RMSE is 0.1870

In [61]:
# mape
np.mean(np.abs((y_true - y_pred) / y_true))

0.13280367888673655